<a href="https://colab.research.google.com/github/fiocruzvig/mortalidade_tuberculose_SIM/blob/main/TuberculoseSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download das biblioteca do PySUS
!pip install pysus

In [48]:
from pysus.ftp.databases.sim import SIM
from google.colab import drive
drive.mount('/content/drive')
import os
import time
import pandas as pd
from openpyxl import load_workbook, Workbook
#Em caso de falha na biblioteca openpyxl, usar o comando "!pip install --upgrade openpyxl"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
sim = SIM().load() # Loads the files from DATASUS
sim.metadata

{'long_name': 'Sistema de Informação sobre Mortalidade',
 'source': 'http://sim.saude.gov.br',
 'description': ''}

In [11]:
sim.groups

{'CID10': 'DO', 'CID9': 'DOR'}

In [34]:
sim.paths

(/dissemin/publicos/SIM/CID10/DORES, /dissemin/publicos/SIM/CID9/DORES)

In [35]:
lista_2022 = sim.get_files("CID10", year=2022)
lista_2021 = sim.get_files("CID10", year=2021)
lista_2020 = sim.get_files("CID10", year=2020)
lista_2019 = sim.get_files("CID10", year=2019)

In [36]:
#Exemplo do Dataframe especificando somente o Rio de Janeiro.
RJ_2021 = sim.get_files("CID10", uf="RJ", year=2021)

parquet = RJ_2021[0].download()
df = parquet.to_dataframe()
df.to_csv("/content/TUB_MORT_RJ.csv")

In [37]:
def percorre_lista(lista):
    for dado in lista:
      uf = str(dado)[2:4]
      if uf == "BR":
        continue
      ano =str(dado)[4:8]
      baixa_DBSIM(int(ano), uf)

In [38]:
def baixa_DBSIM(ano, uf):
    if not os.path.isdir(f'/content/{ano}'):
      os.mkdir(f'/content/{ano}')
    dado = sim.get_files("CID10", uf=uf, year=ano)
    dado = dado[0]
    sigla = str(dado)[2:4]
    ano = str(dado)[4:8]

    if not os.path.isfile(f'/content/{ano}/TUB_MORT_{sigla}.csv'):
      print(f"{ano}: {sigla}: Baixando..")
      print("##########")
      parquet = dado.download()
      df = parquet.to_dataframe()
      df.to_csv(f'/content/{ano}/TUB_MORT_{sigla}.csv')

In [ ]:
#Exercute o código abaixo para baixar os dados do SIM separados pelos anos somente uma vez
"""percorre_lista(lista_2022)
percorre_lista(lista_2021)
percorre_lista(lista_2020)
percorre_lista(lista_2019)"""

In [40]:
#Cria uma lista com classificação CID10 para os casos de tuberculose
def obtem_causas_tub():
    lista_CAUSABAS = []
    for i in range(15, 20):
      for j in range(0, 10):
        lista_CAUSABAS.append(f'A{i}{j}')

    return lista_CAUSABAS

> CID10: Classificação Estatística Internacional de Doenças e Problemas Relacionados à Saúde
>
> CAUSABAS_O: Causa básica original, a primeira informação que entra nosistema
>
> CAUSABAS: Causa básica, conforme a Classificação Internacional de Doença (CID), 10a. Revisão
>
> Retirado de https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacoes-de-mortalidade-sim/dicionario-de-variaveis/ em 23/09/2024

In [41]:
#Leitura do dados da polulação oriundos do ibge
def obtem_df_ibge(arquivo):
    df_ibge = pd.read_excel(arquivo)
    df_ibge = df_ibge[["CODIGO_UF", "POP_ANT"]]
    df_ibge = df_ibge.dropna()
    df_ibge = df_ibge.sort_values(by=["CODIGO_UF"])
    return df_ibge

In [42]:
#Cria a lista de acordo a sigla UF
def cria_lista_uf(estado, lista_ufs):
  if estado is None or len(estado) == 0:
      lista_ufs = lista_ufs
  elif isinstance(estado, list):
      lista_ufs = estado
  else:
      lista_ufs = []
      lista_ufs.append(estado)
  return lista_ufs

In [43]:
#Filtragem dos anos
def cria_lista_anos(anos):
  lista_anos =[]
  if anos is None or len(anos) == 0:
      lista_anos = [2022, 2021, 2020, 2019]
  elif isinstance(anos, list):
      lista_anos = anos
  else:
      lista_anos.append(anos)
  return lista_anos

>Baixe os [dados do ibge](https://docs.google.com/spreadsheets/d/1cYoWKFGNAhVH7_6ER8KQL1CjqpcK0C_X/edit?usp=sharing&ouid=106456478599498612489&rtpof=true&sd=true) usados no projeto e coloque no drive.

In [50]:
#Função principal para a criação da planilha com coeficiente de Mortalidade por tuberculose
def cria_dt(estado=None, anos=None):
  lista_CAUSABAS = obtem_causas_tub()
  df_ibge = obtem_df_ibge("/content/dados_ibge_projecao_tab_net_2022.xlsx")
  lista_ufs = cria_lista_uf(estado, df_ibge["CODIGO_UF"].to_list())
  lista_anos = cria_lista_anos(anos)

  if not os.path.isfile("/content/TUB_MORT_GERAL.xlsx"):
    wb = Workbook()
    wb.save("/content/TUB_MORT_GERAL.xlsx")

  excel_workbook = load_workbook("/content/TUB_MORT_GERAL.xlsx")
  writer = pd.ExcelWriter("/content/TUB_MORT_GERAL.xlsx", engine='openpyxl')

  for ano in lista_anos:
    df_novo = pd.DataFrame(columns=["UF_SIGLA", "NUM_OBITOS", "POPULACAO", "COF_MORTE_CEM_MIL", "ANO"])
    for uf in lista_ufs:
      if os.path.isfile(f'/content/{ano}/TUB_MORT_{uf}.csv'):
          df = pd.read_csv(f'/content/{ano}/TUB_MORT_{uf}.csv', low_memory=False)
          df = df.reset_index()

          df = df[df['CAUSABAS'].isin(lista_CAUSABAS)]
          qtd_obt = df["CAUSABAS"].count()

          populacao = df_ibge[df_ibge["CODIGO_UF"] == uf]["POP_ANT"].values[0]
          coef_mort = (qtd_obt/populacao)*100000
          lista_row = [uf, qtd_obt,populacao,coef_mort,ano]
          df_novo.loc[len(df_novo)] = lista_row


    df_novo = df_novo.round({'COF_MORTE_CEM_MIL': 2})
    df_novo.to_excel(writer, sheet_name=f'{ano}', index=False)

  writer.close()

cria_dt()

In [51]:
df = pd.read_excel("/content/TUB_MORT_GERAL.xlsx", sheet_name="2022")
df

,UF_SIGLA,NUM_OBITOS,POPULACAO,COF_MORTE_CEM_MIL,ANO
0,AC,32,830018,3.86,2022
1,AL,84,3127683,2.69,2022
2,AM,218,733759,29.71,2022
3,AP,19,3941613,0.48,2022
4,BA,413,14141626,2.92,2022
5,CE,229,8794957,2.60,2022
6,DF,28,2817381,0.99,2022
7,ES,115,3833712,3.00,2022
8,GO,111,7056495,1.57,2022
9,MA,217,6776699,3.20,2022
